## Regression

Train and evaluate the baselines. Baselines are: 

- Linear Regression
- L1 regression (Lasso Regression)
- Decision Tree 
- Random Forests 

For every baseline above (except L1), we'll also consider a PCA version where we first reduce the dimensionality. 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

METRICS_DIR = "../outputs/metrics/proxy_metrics_20220426"

# ---------------------------------------------------------------
# DATA LOADING
# ---------------------------------------------------------------
TRAIN_FILEPATH = f"{METRICS_DIR}/train_all_datasets_metrics.csv.gz"
DEV_FILEPATH = f"{METRICS_DIR}/dev_all_datasets_metrics.csv.gz"

train_df = pd.read_csv(TRAIN_FILEPATH, index_col=0)
print("Read dataset with", len(train_df), "examples from", TRAIN_FILEPATH)

dev_df = pd.read_csv(DEV_FILEPATH, index_col=0)
print("Read (holdout) dataset with", len(dev_df), "examples from", DEV_FILEPATH)

# ---------------------------------------------------------------
# FEATURES and TARGET 
# ---------------------------------------------------------------
METRIC_COLS = train_df.select_dtypes("number").columns
features = list(METRIC_COLS[2:])

# Target column will be the normalized human correctness
target = METRIC_COLS[1]
print("Features:\t", features, "\n\nTarget:\t", target)

Read dataset with 31069 examples from ../outputs/metrics/proxy_metrics_20220426/train_all_datasets_metrics.csv.gz
Read (holdout) dataset with 4009 examples from ../outputs/metrics/proxy_metrics_20220426/dev_all_datasets_metrics.csv.gz
Features:	 ['exact_match', 'meteor', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum', 'bleurt', 'bert-score', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'bleu-precision0', 'bleu-precision1', 'bleu-precision2', 'bleu-precision3', 'precision', 'recall', 'f1_score', 'csi', 'num_edits', 'edit_score'] 

Target:	 human_correctness


In [ ]:
# TODO 
# - [] Fit linear regression
# - [] Fit L1 regression

TODO 
- [ ] Compute Baselines (Avg metric)
- [ ] Compute Linear Regression 
- [ ] L1 regression
- [ ] Decision Tree
- [ ] Random Forest
- 

###  Average Baseline

In [ ]:
class AvgBaseline:
    def fit(self, X, y):
        pass
    
    def predict(self, X):
        y_pred = np.mean(X, axis=1)
        assert len(y_pred) == X.shape[0]
        return y_pred

In [ ]:
AvgBaseline().predict(dev_df[features])

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

import sklearn.metrics as metrics
import scipy.stats as st

import os
import joblib


EXPERIMENT_DIR = "../outputs/experiment"
print("Persisting experiment results at", EXPERIMENT_DIR)
os.makedirs(EXPERIMENT_DIR, exist_ok=True)

### Fit models

In [38]:
class Pipeline:
    def __init__(self, model_class, model_hparams, dataset, features, target, seed=81263):
        self.model_class = model_class
        self.model_hparams = model_hparams
        self.dataset = dataset
        self.features = features
        self.target = target
        self.seed = seed
        
    def load_data(self, data):
        """"""
        data = data[data["dataset"] == self.dataset].copy()
        self.X_train = data[self.features]
        self.y_train = data[self.target]
        
    def split(self, holdout_fraction=0.2):
        """"""
        from sklearn.model_selection import train_test_split

        X_train, X_test, y_train, y_test = train_test_split(
            self.X_train, self.y_train,
            test_size=holdout_fraction, 
            random_state=self.seed, 
            stratify=self.y_train,
        )
        
        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test        
        
    def preprocess(self, with_std=True, with_pca=False, **kwargs):
        """"""
        from sklearn.preprocessing import StandardScaler
        from sklearn.pipeline import make_pipeline
        from sklearn.decomposition import PCA
        
        operations = []
        
        if with_std:
            operations.append(StandardScaler())
        if with_pca:
            operations.append(PCA(random_state=self.seed, **kwargs))
        
        self.preproc_fn = make_pipeline(*operations)
        self.preproc_fn.fit(self.X_train)
        self.X_train = self.preproc_fn.transform(self.X_train)
        self.X_test = self.preproc_fn.transform(self.X_test)
        
    def fit(self):
        """"""
        self.model = self.model_class(**self.model_hparams)
        self.model.fit(self.X_train, self.y_train)
        
    def evaluate(self, eval_dataset=None):
        """"""
        import sklearn.metrics as m
        import scipy.stats as st
        
        if eval_dataset is None:
            print("Evaluating holdout dev set.")
            X_test, y_test = self.X_test, self.y_test
        else:
            X_test = eval_dataset[self.features]
            y_test = eval_dataset[self.target]    
            X_test = self.preproc_fn.transform(X_test)
        
        # Evaluation
        scores = self.model.predict(self.X_test)
        
        return {
            "trained_on": self.dataset,
            "mse": m.mean_squared_error(y_pred=scores, y_true=y_test),
            "r2": m.r2_score(y_pred=scores, y_true=y_test),
            "pearson": st.pearsonr(scores, y_test)[0],
            "spearman": st.spearmanr(scores, y_test)[0],
        }

In [39]:
from sklearn.linear_model import LinearRegression

lr_pipeline = Pipeline(LinearRegression, {}, "drop", features, target)
lr_pipeline.load_data(train_df)
lr_pipeline.split(holdout_fraction=0.2)
lr_pipeline.preprocess(with_std=True)
lr_pipeline.fit()
lr_pipeline.evaluate()

Evaluating holdout dev set.


{'trained_on': 'drop',
 'mse': 0.07268463638902901,
 'r2': 0.4327169015577803,
 'pearson': 0.6643839861546849,
 'spearman': 0.6749595512125868}

In [32]:
train_df[train_df.dataset == "drop"]

,example_id,reference,candidate,reference_tokens,candidate_tokens,human_correctness_original,human_correctness,exact_match,meteor,rouge1,...,bleu-precision2,bleu-precision3,precision,recall,f1_score,csi,num_edits,edit_score,dataset,split
5033,006b6b4714b9e67fd29410d9bae9cdad,"diabetes, hypertension",hypertension,"['diabetes', ',', 'hypertension']",['hypertension'],3,0.50,0.0,0.178571,0.666667,...,0.0,0.0,1.000000,0.333333,0.500000,0.333333,1,0.500000,drop,train
5034,009943eb797324320731ac41dcb30e4b,"under the age of 18, 25 to 44, 45 to 64",25 to 44,"['under', 'the', 'age', 'of', '18', ',', '25',...","['25', 'to', '44']",2,0.25,0.0,0.125000,0.428571,...,1.0,0.0,1.000000,0.230769,0.375000,0.230769,9,0.818182,drop,train
5035,011a8c8c2a833ca11db9246f11403a2a,6-yard touchdown pass,drew brees 6 - yard touchdown pass,"['6', '-', 'yard', 'touchdown', 'pass']","['drew', 'brees', '6', '-', 'yard', 'touchdown...",5,1.00,0.0,0.551471,0.800000,...,0.6,0.5,0.714286,1.000000,0.833333,0.714286,5,1.666667,drop,train
5036,0165c8d4a359cbbeedf4ce61ed6f1760,"60 and 69, 70 to 79, 80 to 89, 90 to 99",Cunter,"['60', 'and', '69', ',', '70', 'to', '79', ','...",['Cunter'],1,0.00,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,12,1.000000,drop,train
5037,01b38514f642d6477fec2b1d45f4bf01,born outside the EU,outside the eu,"['born', 'outside', 'the', 'EU']","['outside', 'the', 'eu']",5,1.00,0.0,0.754986,0.857143,...,0.0,0.0,0.666667,0.500000,0.571429,0.400000,1,0.250000,drop,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5715,fefc92db527b53e1a6a5b762a5deb6ae,a pump boat in Mususiasi,Semporna,"['a', 'pump', 'boat', 'in', 'Mususiasi']",['Semporna'],1,0.00,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,5,1.000000,drop,train
5716,ff52abd7769592ff86e4540fcf2d4b7f,Romania,treaty of bucharest,['Romania'],"['treaty', 'of', 'bucharest']",1,0.00,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3,3.000000,drop,train
5717,ff5363a23aeb6ff0cb8a920d12047185,the Algerian War of Independence,"six - day war by israel ( 1967 ) , in nigeria ...","['the', 'Algerian', 'War', 'of', 'Independence']","['six', '-', 'day', 'war', 'by', 'israel', '('...",1,0.00,0.0,0.081301,0.075472,...,0.0,0.0,0.012821,0.200000,0.024096,0.012195,76,15.200000,drop,train
5718,ff6a2ddb643665bbd25097c4e58246f5,"Lance Berkman, Brad Ausmus, Chris Burke",Albert Pujols,"['Lance', 'Berkman', ',', 'Brad', 'Ausmus', ',...","['Albert', 'Pujols']",1,0.00,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,6,1.000000,drop,train


In [ ]:
dataset_name = "drop"
dataset = TRAIN_DATASETS[dataset_name]

# preprocess dataset
data = train_test_split(dataset[features], dataset[target], preprocessor, test_fraction=0.2, seed=SEED)

# fit model
model, model_results = fit_model(AvgBaseline, data)

# evaluate in validation
model_results


In [ ]:
for dataset_name, dataset in TRAIN_DATASETS.items():
    # Dataset preprocessor
    preprocessor = scaler()
    
    data = train_test_split(dataset[features], dataset[target], preprocessor, test_fraction=0.2, seed=SEED)
    model, model_results = fit_model(AvgBaseline, data, features=features[1:])
    model_results["dataset"] = dataset_name
    model_results["seed"] = SEED
    model_results["test_fraction"] = 0.2
    model_results["preprocessing"] = "StandardScaler"
    results.append(model_results)

In [ ]:
def fit_model(model_class, data: tuple, **kwargs) -> tuple:
    X_train, X_test, y_train, y_test = data
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    # Estimator
    model = model_class(**kwargs)
    model.fit(X_train, y_train)

    # Evaluate in holdout set
    scores = model.predict(X_test)
    results = {
        "mse": metrics.mean_squared_error(y_true=y_test, y_pred=scores),
        "r2": metrics.r2_score(y_true=y_test, y_pred=scores),
        "pearson": st.pearsonr(scores, y_test)[0],
        "spearman": st.spearmanr(scores, y_test)[0],
    }

    return model, results

In [ ]:
def compute_m

In [ ]:
from sklearn.linear_model import LinearRegression 

train_datasets = TRAIN_DATASETS
preproc_fn = scaler
test_fraction = 0.2
seed = seed

model_selection_results = []
for dataset_name, dataset in train_datasets.items():
    # Dataset preprocessor
    preprocessor = preproc_fn()
    
    data = train_test_split(dataset[features], dataset[target], preprocessor, test_fraction=test_fraction, seed=SEED)
    model, model_results = fit_model(LinearRegression, data)
    model_results["dataset"] = dataset_name
    model_results["seed"] = SEED
    model_results["test_fraction"] = test_fraction
    model_results["preprocessing"] = "StandardScaler"
    results.append(model_results)

In [ ]:
pd.DataFrame(results)

In [ ]:
results

In [ ]:




def fit_model(data, estimator, dataset=None):

    if dataset is not None:
        data = data[data["dataset"] == dataset]
    
    print("Considering dataset with", len(data), "examples, spanning datasets:", data.dataset.unique())
    X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.20, random_state=78452, stratify=data[target])
    print(X_train.shape, X_test.shape)
    
    # Preprocessing data (since LR may be sensitive to it)
    X_train_prec, scalers = preprocess(X_train)
    X_test_prec, _ = preprocess(X_test, scalers=scalers)

    # Create estimator
    clf = estimator()
    clf.fit(X_train_prec, y_train)

    # Evaluate
    scores = clf.predict(X_test_prec)
    results = {
        "mse": metrics.mean_squared_error(y_test, scores),
        "r2": metrics.r2_score(y_test, scores),
        "pearson": pearsonr(scores, y_test)[0],
        "spearman": spearmanr(scores, y_test),
    }
    return clf, scalers, results


def eval_datasets(model, eval_datasets: dict, scalers: dict):
    eval_results = {}
    eval_scores = {}
    for dataset_name, dataset in eval_datasets.items():
        X, y = dataset[features], dataset[target]

        X_prec, _ = preprocess(X.copy(), scalers=scalers)

        scores = model.predict(X_prec)
        eval_results[dataset_name] = {
            "mse": metrics.mean_squared_error(y, scores),
            "r2": metrics.r2_score(y, scores),
            "pearson": pearsonr(scores, y)[0],
            "spearman": spearmanr(scores, y)[0],
        }
        eval_scores[dataset_name] = scores
        
    return eval_results, eval_scores


In [ ]:
from sklearn.linear_model import LinearRegression

# Sanity check
lr, lr_scalers, valid_results = fit_model(train_df, LinearRegression, dataset="narrativeqa")
valid_results

In [ ]:
dev_df[features]

In [ ]:
from scipy.stats import pearsonr, spearmanr

    
    
model = AvgBaseline(features[1:])
print("Using metrics:", model._features)
avg_metrics = model.predict(dev_df)

from sklearn.metrics import mean_squared_error, mean_absolute_error
print("MSE:", mean_squared_error(y_pred=avg_metrics, y_true=dev_df[target]))
print("MAE:", mean_absolute_error(y_pred=avg_metrics, y_true=dev_df[target]))
print("Pearson:", pearsonr(avg_metrics, dev_df[target])[0])
print("Spearman:", spearmanr(avg_metrics, dev_df[target])[0])

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
print("MSE:", mean_squared_error(y_pred=avg_metrics, y_true=dev_df[target]))
print("MAE:", mean_absolute_error(y_pred=avg_metrics, y_true=dev_df[target]))
print("Pearson:", pearsonr(avg_metrics, dev_df[target])[0])
print("Spearman:", spearmanr(avg_metrics, dev_df[target])[0])

In [ ]:
from collections import defaultdict

# Unique datasets
unique_datasets = list(train_df.dataset.unique())

# Evaluation datasets
# includes all_datasets (macro eval), as well as individual datasets
dev_orig_datasets = {None: dev_df}
dev_orig_datasets.update({dataset: dev_df[dev_df.dataset == dataset] for dataset in unique_datasets})

models = {}
results_by_dataset = {}
for dataset_name in dev_orig_datasets.keys():
    print("Fitting model using", "all" if dataset_name is None else dataset_name, "datasets")
    model, model_scalers, valid_results = fit_model(train_df, LinearRegression, dataset=dataset_name)
    
    models[dataset_name] = model
    results, scores = eval_datasets(model, dev_orig_datasets, model_scalers)
    
    results_by_dataset[dataset_name] = results
    
    
    


def parse_table_results(results_by_dataset, filename, output_dir=METRICS_DIR):
    table_results = defaultdict(list)

    for train_dataset, test_values in results_by_dataset.items():

        for test_dataset, test_results in test_values.items():
            table_results["train_dataset"].append("all_datasets" if train_dataset is None else train_dataset)
            table_results["eval_dataset"].append("all_datasets" if test_dataset is None else test_dataset)

            for metric, metric_value in test_results.items():
                table_results[metric].append(metric_value)
            
    table_results = pd.DataFrame(table_results)
    table_results.to_csv((f"{output_dir}/{filename}.csv"))
    return table_results


table_results = parse_table_results(results_by_dataset, output_dir=METRICS_DIR, filename="dev_lr_correlations")
table_results

In [ ]:
def plot_model_coeffs(models, train_dataset): 
    clf = models[train_dataset]
    plt.figure(figsize=(10, 5))
    plt.title(f"Feature importance for train dataset: {train_dataset if train_dataset is not None else 'all_datasets'}")
    sns.barplot(y=features, x=clf.coef_, orient="h")
    plt.xlim(-1, 1)
    plt.show()
    

for train_dataset in models.keys():
    plot_model_coeffs(models, train_dataset)


### Leave-one-out (LOO) Regression

In [ ]:
from collections import defaultdict

# Unique datasets
_unique_datasets = list(train_df.dataset.unique())

# Evaluation datasets
# includes all_datasets (macro eval), as well as individual datasets
_dev_orig_datasets = {None: dev_df}
_dev_orig_datasets.update({dataset: dev_df[dev_df.dataset == dataset] for dataset in unique_datasets})

_models = {}
_results_by_dataset = {}
for _dataset_name in _dev_orig_datasets.keys():
    if _dataset_name is None: continue
    
    # Compute other dataset names except `_dataset_name`
    _remaining_datasets = [k for k in _dev_orig_datasets.keys() if k != _dataset_name]
    
    # Select subset of trainin data that does not include `_dataset_name`
    _train_remain_df = train_df[train_df.dataset.isin(_remaining_datasets)]
    _train_remain_name = f"all_except_{_dataset_name}"
    
    print("Fitting model on", _train_remain_name, f"with {len(_train_remain_df)} examples (instead of {len(train_df)})")

    _model, _model_scalers, _valid_results = fit_model(_train_remain_df, LinearRegression)
    _models[_train_remain_name] = _model
    _results, _scores = eval_datasets(_model, _dev_orig_datasets, _model_scalers)
    _results_by_dataset[_train_remain_name] = _results
    

parse_table_results(_results_by_dataset, "dev_lr_loo_correlations", METRICS_DIR)

In [ ]:
for train_dataset in _models.keys():
    plot_model_coeffs(_models, train_dataset)

### Principal Component Analysis

We've seen there is a large correlation between the different metrics.. In particular, it might explain the coefficients, we see in the image above. In the presence of redundancy, the model may be [non-identifiable](https://en.wikipedia.org/wiki/Identifiability), i.e., have two or more parameterizations that are observationally equivalent.

In this section of the notebook, we are interested in knowing whether there will be a set of orthogonal components that fully explain the model. 

In [ ]:
def eval_datasets_with_pca(model, eval_datasets: dict, pca: dict):
    eval_results = {}
    eval_scores = {}
    for dataset_name, dataset in eval_datasets.items():
        X, y = dataset[features], dataset[target]

        X_prec = pca.transform(X.copy())

        scores = model.predict(X_prec)
        eval_results[dataset_name] = {
            "mse": metrics.mean_squared_error(y, scores),
            "r2": metrics.r2_score(y, scores),
            "pearson": pearsonr(scores, y)[0],
            "spearman": spearmanr(scores, y)[0],
        }
        eval_scores[dataset_name] = scores
    return eval_results, eval_scores

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
data = train_df.copy()
print("Considering dataset with", len(data), "examples, spanning datasets:", data.dataset.unique())

X_train, X_test, y_train, y_test = train_test_split(
    data[features], data[target], test_size=0.20, random_state=78452, stratify=data[target])
print(X_train.shape, X_test.shape)
    
# Preprocessing data (since LR may be sensitive to it)
# X_train_prec, scalers = preprocess(X_train)
# X_test_prec, _ = preprocess(X_test, scalers=scalers)


# Iterate over several components
eval_results = defaultdict(list)
_pca_models = {}
_pca = {}
for n in range(2, 20):
    print(X_train.shape, X_test.shape)
    print("Fitting PCA w/ n_components =", n)
    for seed in (123124, 1295532, 875843):
        # Create estimator
        pca = PCA(n_components=n, random_state=seed)
        X_train_transf = pca.fit_transform(X_train.copy())
        X_test_transf = pca.transform(X_test.copy())
        # print(X_train_transf.shape, X_test_transf.shape)

        # Fit LR on top of new representation
        lr = LinearRegression()
        lr.fit(X_train_transf, y_train)

        scores = lr.predict(X_test_transf)
        eval_results["n"].append(n)
        eval_results["seed"].append(n)
        eval_results["mse"].append(metrics.mean_squared_error(y_test, scores))
        eval_results["r2"].append(metrics.r2_score(y_test, scores))
        eval_results["pearson"].append(pearsonr(scores, y_test)[0])
        eval_results["spearman"].append(spearmanr(scores, y_test)[0])
        
        _pca[(n, seed)] = pca
        _pca_models[(n, seed)] = lr
        
eval_results = pd.DataFrame(eval_results)

In [ ]:
sns.lineplot(data=eval_results, x="n", y="mse")
plt.xlabel("N components (PCA)")
plt.title("MSE of fit in function of number of PCA components on dev set")
plt.xlim(0, 18)
plt.ylim(0.08, 0.15)
plt.show()

In [ ]:
n_components, seed = 10, 1295532
model = _pca_models[(n_components, seed)]

# Unique datasets
unique_datasets = list(train_df.dataset.unique())

# Evaluation datasets
# includes all_datasets (macro eval), as well as individual datasets
dev_orig_datasets = {None: dev_df}
dev_orig_datasets.update({dataset: dev_df[dev_df.dataset == dataset] for dataset in unique_datasets})

results_by_dataset = {}
for dataset_name in dev_orig_datasets.keys():
    print("Fitting model using", "all" if dataset_name is None else dataset_name, "datasets")
    results, scores = eval_datasets_with_pca(
        model=model,
        pca=_pca[(n_components, seed)], 
        eval_datasets=dev_orig_datasets,
    )
    
    results_by_dataset[dataset_name] = results 

parse_table_results(results_by_dataset, f"dev_pca_{n_components}+lr_correlations", METRICS_DIR)

In [ ]:

# Reduce dimension to 2 with PCA
pca = {i: lambda: make_pipeline(StandardScaler(), PCA(n_components=i, random_state=SEED)) for i in range(2, 15)}